In [1]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


# Instalations

In [2]:
!pip install -qU langchain-ollama
! pip install ollama
! curl https://ollama.ai/install.sh | sh

  % Total    % Received % Xferd  Average Speed   Time    Time     Time  Current
                                 Dload  Upload   Total   Spent    Left  Speed
100 13281    0 13281    0     0  52150      0 --:--:-- --:--:-- --:--:-- 52287
>>> Installing ollama to /usr/local
>>> Downloading Linux amd64 bundle
######################################################################## 100.0%
>>> Creating ollama user...
>>> Adding ollama user to video group...
>>> Adding current user to ollama group...
>>> Creating ollama systemd service...
>>> The Ollama API is now available at 127.0.0.1:11434.
>>> Install complete. Run "ollama" from the command line.


In [3]:

!sudo apt-get update
!sudo apt-get install pciutils lshw


Get:1 https://cli.github.com/packages stable InRelease [3,917 B]
Get:2 https://cloud.r-project.org/bin/linux/ubuntu jammy-cran40/ InRelease [3,632 B]
Get:3 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  InRelease [1,581 B]
Get:4 https://cli.github.com/packages stable/main amd64 Packages [345 B]
Get:5 https://r2u.stat.illinois.edu/ubuntu jammy InRelease [6,555 B]
Hit:6 http://archive.ubuntu.com/ubuntu jammy InRelease
Get:7 http://security.ubuntu.com/ubuntu jammy-security InRelease [129 kB]
Get:8 https://developer.download.nvidia.com/compute/cuda/repos/ubuntu2204/x86_64  Packages [2,227 kB]
Get:9 http://archive.ubuntu.com/ubuntu jammy-updates InRelease [128 kB]
Get:10 https://r2u.stat.illinois.edu/ubuntu jammy/main amd64 Packages [2,860 kB]
Hit:11 https://ppa.launchpadcontent.net/deadsnakes/ppa/ubuntu jammy InRelease
Get:12 https://r2u.stat.illinois.edu/ubuntu jammy/main all Packages [9,573 kB]
Hit:13 https://ppa.launchpadcontent.net/graphics-drivers/ppa/ubun

In [4]:
!pip install statistics scipy

  Preparing metadata (setup.py) ... done
  Created wheel for statistics: filename=statistics-1.0.3.5-py3-none-any.whl size=7435 sha256=83355f2786d399aba2f229b956d61ab81f074eeedb0f36650d41802ce180f853
  Stored in directory: /root/.cache/pip/wheels/10/9c/1a/0c68a89e4533a18e9f9902018b94ffc2481139818b67cbb05a
Successfully built statistics


In [5]:
!pip install icecream

# Running Ollama Server on colab

In [6]:
import os
import threading
import subprocess
import requests
import json
import pandas as pd

def ollama():
    os.environ['OLLAMA_HOST'] = '0.0.0.0:11434'
    os.environ['OLLAMA_ORIGINS'] = '*'
    subprocess.Popen(["ollama", "serve"])


ollama_thread = threading.Thread(target=ollama)
ollama_thread.start()

In [7]:
# pull the selected model
!ollama pull gpt-oss:20b

# Data Loading

In [8]:
SAMPLE_JSON_FILE = "/content/drive/MyDrive/WSD_dataset/sample_data.json"
DEV_JSON_FILE = "/content/drive/MyDrive/WSD_dataset/train.json"
TRAIN_JSON_FILE = "/content/drive/MyDrive/WSD_dataset/dev.json"
def load_data(file_path):
    """
    Loads the json containing the dataset and return a pandas dataframe.
    """
    with open(file_path, 'r') as f:
        data = json.load(f)
    # Transpose because the json is {id: {features...}, ...}
    df = pd.DataFrame(data).T

    # Ensure 'average' is float
    df['average'] = df['average'].astype(float)
    # Ensure 'choices' is list (for scoring later)
    return df

df_sample = load_data(SAMPLE_JSON_FILE)
df_dev = load_data(DEV_JSON_FILE)
df_train = load_data(TRAIN_JSON_FILE)

# Test The ollama model

In [9]:
from ollama import chat
from ollama import ChatResponse
from pydantic import BaseModel
import subprocess
import gc
import pandas as pd
import json
from icecream import ic
import os
from tqdm import tqdm
import time

MODEL_NAME = 'gpt-oss:20b'
# Random element in the dataset

class Score(BaseModel):
    score: int

# The next class is just for testing
class City(BaseModel):
    city_name: str


In [10]:
SYSTEM_PROMPT = (
    """
    You are an expert NLU annotator. Your job is to rate how plausible a candidate meaning (sense)
    is for the HOMONYM used in the target sentence within the short story.

    Return ONLY a single JSON object with one key: "score" and an integer value 1, 2, 3, 4 or 5.
    Integer mapping:
      1 = Definitely not
      2 = Probably not
      3 = Ambiguous / Unsure
      4 = Probably yes
      5 = Definitely yes

    The response must be a JSON object and nothing else, for example: {{"score": 4}}
    """
)

USER_PROMPT = (
    """
    [STORY]
    {full_story_text}

    [HOMONYM]
    {homonym}

    [CANDIDATE SENSE]
    {sense_text}

    [TASK]
    Based on the STORY above, decide how plausible it is that the HOMONYM is used with the
    CANDIDATE SENSE in the target sentence.

    Return ONLY a single JSON object with one key "score" and an integer value (1-5)
    as described by the system message. Example output: {{"score": 3}}
    """
)

In [11]:
def create_full_story_text(item):
    """Compose the story text used as context for rating.

    Uses `precontext`, `sentence`, and `ending` fields when available and joins them into a single string.
    """
    fullstory = f"{item.get('precontext', '')} {item.get('sentence', '')} {item.get('ending', '')}"
    return fullstory.strip()


def create_message(item):
    sense = f"{item.get('judged_meaning', '')} as in \"{item.get('example_sentence', '')}\"".strip()
    homonym = item.get("homonym", "")
    full_story_text = create_full_story_text(item)

    messages = [
        {"role": "system", "content": SYSTEM_PROMPT},
        {"role": "user", "content": USER_PROMPT.format(
            full_story_text=full_story_text,
            homonym=homonym,
            sense_text=sense,
        )},
    ]
    return messages

In [12]:
# Random element in the dataset
item = df_sample.iloc[0].to_dict()

messages = create_message(item)

#model_number = 6  # no use for this here

response: ChatResponse = chat(model=MODEL_NAME,
                              messages=messages,
                              think="high",
                              format=Score.model_json_schema(),
                              options={
                                  "temperature": 0.8
                              }
                              )

# ic(messages)
ic(response.model)
ic(response.total_duration * 10e-9)  # convert from ns to s
ic(response.message.role)
ic(response.message.content)
ic(response.message.thinking)
pass

ic| response.model: 'gpt-oss:20b'
ic| response.total_duration * 10e-9: 683.18175707
ic| response.message.role: 'assistant'
ic| response.message.content: '{"score": 5}'
ic| response.message.thinking: '''We need to analyze the prompt carefully.
                                
                                We have a story about Anna having a tough week, her computer crashing, frustrated, she called Jen, asked friend to help her get rid of the bugs. They were crawling on the keyboard. Maybe that was the reason it didn't work.
                                
                                The homonym: "bugs". The candidate sense: "general term for any insect or similar creeping or crawling invertebrate as in 'The garden was full of bugs.'"
                                
                                We need to decide how plausible it is that "bugs" is used with that candidate sense in the target sentence. The target sentence presumably is the sentence containing "bugs" in the story

In [13]:
# MODEL_NAME = 'gpt-oss:20b'
# n_candidates = 3

# def llm_self_evaluated_prediction(item,n_candidates=3,model = MODEL_NAME):
#     # --- STAGE 1: Generate 3 Candidates ---
#     messages = create_message(item)
#     candidates = []

#     print(f"Generating {n_candidates} candidates using {MODEL_NAME}...")
#     for i in range(n_candidates):
#         response = chat(
#             model=model,
#             messages=messages,
#             think="low",
#             options={"temperature": 0.8}
#         )

#         candidates.append({
#             "index": i,
#             "score_json": response.message.content,
#             "reasoning": getattr(response.message, 'thinking', "No thinking provided")
#         })

#     # --- STAGE 2: Self-Selection ---
#     # We build a new prompt to let the model "look in the mirror"
#     candidates_summary = "\n".join([
#         f"CANDIDATE {c['index']}:\nResult: {c['score_json']}\nReasoning: {c['reasoning']}\n"
#         for c in candidates
#     ])

#     selection_prompt = f"""
#     You are an expert NLU judge. Below are 3 independent analyses for the same linguistic task.
#     Review the story and the reasoning provided by each candidate, then select the one that is most accurate.

#     [STORY]
#     {create_full_story_text(item)}

#     [HOMONYM]: {item.get('homonym')}
#     [CANDIDATE SENSE]: {item.get('judged_meaning')}

#     [CANDIDATE ANALYSES]
#     {candidates_summary}

#     [TASK]
#     Identify which candidate (0, 1, or 2) provided the most logical score and reasoning.
#     Return ONLY a JSON object with the index and the final chosen score.
#     Example: {{"selected_index": 1, "score": 2}}
#     """

#     print(f"Selecting best candidate...")
#     selection_response = chat(
#         model=MODEL_NAME,
#         messages=[{"role": "user", "content": selection_prompt}],
#         format={'type': 'object', 'properties': {
#             'selected_index': {'type': 'integer'},
#             'score': {'type': 'integer'}
#         }}
#     )

#     # --- FINAL OUTPUT ---
#     selection_data = json.loads(selection_response.message.content)
#     best_candidate = candidates[selection_data['selected_index']]

#     return {
#         "final_score": selection_data['score'],
#         "winning_reasoning": best_candidate['reasoning'],
#         "candidate_index": selection_data['selected_index']
#     }

In [14]:
# item = df_sample.iloc[0].to_dict()


In [15]:
# response = llm_self_evaluated_prediction(item)

In [16]:
# response

In [17]:
item

{'homonym': 'bugs',
 'judged_meaning': 'general term for any insect or similar creeping or crawling invertebrate',
 'precontext': 'Anna was having a tough week. Her room was a mess, and her computer kept crashing. Frustrated by everything going wrong, she called Jen.',
 'sentence': 'She asked her friend to help her get rid of the bugs.',
 'ending': "They were crawling on the keyboard. Maybe that was the reason it didn't work.",
 'choices': [5, 5, 1, 2, 5],
 'average': 3.6,
 'stdev': 1.949358868961793,
 'nonsensical': [False, False, False, False, False],
 'example_sentence': 'The garden was full of bugs.'}

# Apply on the dev dataset


In [18]:
def get_dev_predictions(model_name, df,think='low',temp = 0.8, max_examples=None,few_shot_mode=False):
    preds = []
    failed_ids = []
    ids = list(df.index.astype(str))
    if max_examples is not None:
        ids = ids[:max_examples]

    total_start = time.perf_counter()
    per_item_times = []

    for idx in tqdm(ids):
        item = df.loc[idx].to_dict()
        messages = create_message(item)

        start = time.perf_counter()
        try:
            # --- STEP 1: Generate 3 Candidates ---
            candidates = []
            for i in range(3):
                res = chat(
                    model=model_name,
                    messages=messages,
                    think= think,
                    format=Score.model_json_schema(),
                    options={"temperature": temp} # High temp for diversity
                )
                candidates.append({
                    "index": i,
                    "content": res.message.content,
                    "thinking": getattr(res.message, 'thinking', "No reasoning provided")
                })

            # --- STEP 2: Selection Pass ---
            candidates_text = "\n".join([
                f"ID {c['index']}: Result {c['content']} | Reasoning: {c['thinking']}"
                for c in candidates
            ])

            selection_prompt = f"""
            Review these 3 NLU annotations for the homonym '{item.get('homonym')}'.
            Select the most logically sound candidate based on the story.

            [STORY]: {create_full_story_text(item)}
            [CANDIDATES]:
            {candidates_text}

            Return ONLY a JSON object with the 'score' of the best candidate.
            Example: {{"score": 2}}
            """

            final_response = chat(
                model=model_name,
                messages=[{"role": "user", "content": selection_prompt}],
                format=Score.model_json_schema(),
                options={"temperature": 0} # Zero temp for stable selection
            )

            elapsed = time.perf_counter() - start
            per_item_times.append((idx, elapsed))

            # --- STEP 3: Validate Final Score ---
            try:
                s = Score.model_validate_json(final_response.message.content)
                pred = int(s.score)
                if not (1 <= pred <= 5):
                    raise ValueError("Score out of range")
            except Exception as e:
                print(f"Validation failed for id {idx}: {e}")
                pred = None
                failed_ids.append(str(idx))

        except Exception as e:
            elapsed = time.perf_counter() - start
            print(f"Error in pipeline for id {idx}: {e}")
            pred = None
            failed_ids.append(str(idx))
            per_item_times.append((idx, elapsed))

        preds.append({"id": str(idx), "prediction": pred, "time": elapsed})

    total_elapsed = time.perf_counter() - total_start
    return {
        "predictions": preds,
        "failed_ids": failed_ids,
        "total_time": total_elapsed,
        "per_item_times": per_item_times,
        "avg_time": sum(t for _, t in per_item_times) / len(per_item_times) if per_item_times else 0,
    }

In [19]:
# for gpt-oss think = True will not have effect , must be low,
model_name = MODEL_NAME
think= 'low'

MAX_EXAMPLES = 100

print(f"\n=== Running model: {model_name} ===")

# If deepseek check if thinking to have different directoies
if think:
    OUT_DIR = f"/content/drive/MyDrive/llm-ollama/zero-shot/{model_name.replace(':', '-')}-think-self-eval"
    os.makedirs(OUT_DIR, exist_ok=True)
else:
    OUT_DIR = f"/content/drive/MyDrive/llm-ollama/zero-shot/{model_name.replace(':', '-')}"
    os.makedirs(OUT_DIR, exist_ok=True)

# get predictions (may take a while if MAX_EXAMPLES is None)
# res = get_dev_predictions(model_name, df_dev, max_examples=MAX_EXAMPLES)
res = get_dev_predictions(model_name, df_dev, max_examples=MAX_EXAMPLES, think=think,few_shot_mode=False)

preds = res["predictions"]

pred_file = os.path.join(OUT_DIR, "predictions.jsonl")
with open(pred_file, "w") as f:
    for p in preds:
        f.write(json.dumps({"id": p["id"], "prediction": p["prediction"]}) + "\n")

# Save failed ids so they can be excluded from scoring
failed_file = os.path.join(OUT_DIR, "failed_ids.jsonl")
with open(failed_file, "w") as f:
    for fid in res.get("failed_ids", []):
        f.write(json.dumps({"id": fid}) + "\n")

# Save timing info and per-item times
timing_file = os.path.join(OUT_DIR, "timing.txt")
with open(timing_file, "w") as f:
    f.write(f"total_time_sec: {res['total_time']:.4f}\n")
    f.write(f"avg_time_sec: {res['avg_time']:.4f}\n")
    f.write("per_item_times_sec:\n")
    for idx, t in res["per_item_times"]:
        f.write(f"{idx}: {t:.4f}\n")

# Create ref.jsonl from df_dev (respect MAX_EXAMPLES) inside the model folder
# Exclude any ids that failed JSON parsing so they won't be evaluated
failed_set = set(res.get("failed_ids", []))
ref_file = os.path.join(OUT_DIR, "ref.jsonl")
with open(ref_file, "w") as f:
    for idx, row in df_dev.iterrows():
        if MAX_EXAMPLES is not None and int(idx) >= MAX_EXAMPLES:
            break
        if str(idx) in failed_set:
            # skip items that produced invalid JSON for this model
            continue
        f.write(json.dumps({"id": str(idx), "label": row["choices"]}) + "\n")

print(f"Predictions saved to {pred_file}")
print(f"Gold data saved to {ref_file}")
print(f"Timing saved to {timing_file}")

# If there is failed attemp rewrite all of the ids so that they are consecutive. This is needed for the scoring script
if len(res.get('failed_ids', [])) > 0:
    # Rewrite pred_file with consecutive ids
    new_pred_file = os.path.join(OUT_DIR, "predictions_consecutive_ids.jsonl")
    with open(pred_file, "r") as fin, open(new_pred_file, "w") as fout:
        for new_id, line in enumerate(fin):
            obj = json.loads(line)
            obj["id"] = str(new_id)
            fout.write(json.dumps(obj) + "\n")
    pred_file = new_pred_file

    # Rewrite ref_file with consecutive ids
    new_ref_file = os.path.join(OUT_DIR, "ref_consecutive_ids.jsonl")
    with open(ref_file, "r") as fin, open(new_ref_file, "w") as fout:
        for new_id, line in enumerate(fin):
            obj = json.loads(line)
            obj["id"] = str(new_id)
            fout.write(json.dumps(obj) + "\n")
    ref_file = new_ref_file

# Run scoring script for this model outputs
res = subprocess.run(["python", "/content/drive/MyDrive/WSD_dataset/score/scoring.py", ref_file, pred_file, os.path.join(OUT_DIR, "score.json")], capture_output=True, text=True)
print(res.stdout)
if res.stderr:
    print("Scoring STDERR:")
    print(res.stderr)

#
subprocess.run(["ollama", "stop", model_name], check=False)
gc.collect()


=== Running model: gpt-oss:20b ===


100%|██████████| 100/100 [35:44<00:00, 21.45s/it]


Predictions saved to /content/drive/MyDrive/llm-ollama/zero-shot/gpt-oss-20b-think-self-eval/predictions.jsonl
Gold data saved to /content/drive/MyDrive/llm-ollama/zero-shot/gpt-oss-20b-think-self-eval/ref.jsonl
Timing saved to /content/drive/MyDrive/llm-ollama/zero-shot/gpt-oss-20b-think-self-eval/timing.txt
Importing...
Starting Scoring script...
Everything looks OK. Evaluating file /content/drive/MyDrive/llm-ollama/zero-shot/gpt-oss-20b-think-self-eval/predictions.jsonl on /content/drive/MyDrive/llm-ollama/zero-shot/gpt-oss-20b-think-self-eval/ref.jsonl
----------
Spearman Correlation: 0.6789437374780357
Spearman p-Value: 8.273543513437362e-15
----------
Accuracy: 0.66 (66/100)
Results dumped into scores.json successfully.



741

test-think
